In [1]:
import tensorflow as tf
import numpy as np

In [2]:
tf.enable_eager_execution() 

In [5]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state([1, 1, 0, 0], [0.98, 1, 0, 0.51])
print('Final result: ', m.result().numpy())  # Final result: 0.75

Final result:  0.75


# 

In [2]:
y_true = np.array([[2], [1], [0], [3], [0]]).astype(np.int64)
y_true = tf.identity(y_true)

y_pred = np.array([[0.1, 0.2, 0.6, 0.1],
                   [0.8, 0.05, 0.1, 0.05],
                   [0.3, 0.4, 0.1, 0.2],
                   [0.6, 0.25, 0.1, 0.05],
                   [0.1, 0.2, 0.6, 0.1]
                   ]).astype(np.float32)
y_pred = tf.identity(y_pred)

In [3]:
_, m_ap = tf.metrics.sparse_average_precision_at_k(y_true, y_pred, 3)

sess = tf.Session()
sess.run(tf.local_variables_initializer())

stream_vars = [i for i in tf.local_variables()]

tf_map = sess.run(m_ap)
print("TF_MAP",tf_map)

print("STREAM_VARS",(sess.run(stream_vars)))

tmp_rank = tf.nn.top_k(y_pred,3)

print("TMP_RANK",sess.run(tmp_rank))

W0813 09:28:34.108921 4508128704 deprecation.py:323] From <ipython-input-3-b34b6f113efb>:1: sparse_average_precision_at_k (from tensorflow.python.ops.metrics_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use average_precision_at_k instead
W0813 09:28:34.149288 4508128704 deprecation.py:323] From /Users/pengcheng.jia/anaconda3/envs/reco/lib/python3.6/site-packages/tensorflow/python/ops/metrics_impl.py:3066: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


TF_MAP 0.4333333333333333
STREAM_VARS [5.0, 2.1666666666666665]
TMP_RANK TopKV2(values=array([[0.6 , 0.2 , 0.1 ],
       [0.8 , 0.1 , 0.05],
       [0.4 , 0.3 , 0.2 ],
       [0.6 , 0.25, 0.1 ],
       [0.6 , 0.2 , 0.1 ]], dtype=float32), indices=array([[2, 1, 0],
       [0, 2, 1],
       [1, 0, 3],
       [0, 1, 2],
       [2, 1, 0]], dtype=int32))


In [6]:
x = tf.placeholder(tf.float32, [1,2])
#x = tf.constant([0.7, 2.2], dtype=tf.float32)
y = tf.constant([[0, 1]], dtype=tf.float32)
acc, opt = tf.metrics.accuracy(tf.round(x), y)


sess = tf.Session()
sess.run(tf.local_variables_initializer())

stream_vars = [i for i in tf.local_variables()]

print(sess.run(acc, {x:[[0.501, 1.2]]}))
print(sess.run(opt, {x:[[0.501, 1.2]]}))

print(sess.run(acc, {x:[[0.4, 1.2]]}))
print(sess.run(opt, {x:[[0.4, 1.2]]}))

RuntimeError: tf.placeholder() is not compatible with eager execution.

In [9]:
rel = tf.placeholder(tf.int64, [1,3])
rec = tf.constant([[7, 5, 10, 6, 3, 1, 8, 12, 31, 88]], tf.int64) 
precision, update_op = tf.metrics.precision_at_k(rel, rec, 10)

# 当k=10，即对rec = tf.constant([[7, 5, 10, 6, 3, 1, 8, 12, 31, 88]], tf.int64) 所有的样本进行排序，
# 进而在函数中实际运用的是rec中样本数值从大到小排列的索引值 = [9,8,7,2,6,0,3,1,4,5]

sess = tf.Session()
sess.run(tf.local_variables_initializer())

stream_vars = [i for i in tf.local_variables()]
#Get the local variables true_positive and false_positive

print("[PRECSION_1]: ",sess.run(precision, {rel:[[1,5,10]]})) # nan
#tf.metrics.precision maintains two variables true_positives 
#and  false_positives, each starts at zero.
#so the output at this step is 'nan'

print("[UPDATE_OP_1]:",sess.run(update_op, {rel:[[1,5,10]]})) #0.2, [1,5] 在 排序的索引值list里
#when the update_op is called, it updates true_positives 
#and false_positives using labels and predictions.

print("[STREAM_VARS_1]:",sess.run(stream_vars)) #[2.0, 8.0]
# Get true positive rate and false positive rate

print("[PRECISION_1]:",sess.run(precision,{rel:[[1,10,15]]})) # 0.2
#So calling precision will use true_positives and false_positives and outputs 0.2

print("[UPDATE_OP_2]:",sess.run(update_op,{rel:[[1,10,15]]})) #0.15
#the update_op updates the values to the new calculated value 0.15.

print("[STREAM_VARS_2]:",sess.run(stream_vars)) #[3.0, 17.0]

[PRECSION_1]:  nan
[UPDATE_OP_1]: 0.2
[STREAM_VARS_1]: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 8.0]
[PRECISION_1]: 0.2
[UPDATE_OP_2]: 0.15
[STREAM_VARS_2]: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 17.0]
